In [25]:
#Importing dependencies
import pandas as pd
from sqlalchemy import create_engine
import datetime as dt

In [26]:
#Extract CSV insto dataframe
#USA covid-19 csv file

covid19_file="Resources/us_states_covid19_daily.csv"
Usacovid_df = pd.read_csv(covid19_file)
Usacovid_df.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,hospitalized,total,totalTestResults,posNeg,fips,deathIncrease,hospitalizedIncrease,negativeIncrease,positiveIncrease,totalTestResultsIncrease
0,2020/04/22,AK,335.0,11824.0,NaN,39.0,36.0,NaN,NaN,NaN,...,36.0,12159.0,12159.0,12159.0,2,0.0,0.0,1034.0,6.0,1040.0
1,2020/04/22,AL,5465.0,43295.0,NaN,NaN,730.0,NaN,288.0,NaN,...,730.0,48760.0,48760.0,48760.0,1,17.0,31.0,0.0,234.0,234.0
2,2020/04/22,AR,2276.0,27437.0,NaN,97.0,291.0,NaN,NaN,23.0,...,291.0,29713.0,29713.0,29713.0,5,-1.0,0.0,2223.0,49.0,2272.0
3,2020/04/22,AS,0.0,3.0,17.0,NaN,NaN,NaN,NaN,NaN,...,NaN,20.0,3.0,3.0,60,0.0,0.0,0.0,0.0,0.0
4,2020/04/22,AZ,5459.0,51142.0,NaN,664.0,NaN,300.0,NaN,195.0,...,NaN,56601.0,56601.0,56601.0,4,21.0,0.0,1241.0,208.0,1449.0


In [27]:
#Extract CSV insto dataframe
#NYC STOCK exchange file

NYCStock_file="Resources/NYCStock.csv"
stock_df=pd.read_csv(NYCStock_file)
stock_df.head()

,date,symbol,open,close,low,high,volume
0,2020/01/05,WLTW,123.430000,125.839996,122.309998,126.250000,2163600
1,2020/01/06,WLTW,125.239998,119.980003,119.940002,125.540001,2386400
2,2020/01/07,WLTW,116.379997,114.949997,114.930000,119.739998,2489500
3,2020/01/08,WLTW,115.480003,116.620003,113.500000,117.440002,2006300
4,2020/01/11,WLTW,117.010002,114.970001,114.089996,117.330002,1408600


In [28]:
#Transforming the USA COVID 19 dataframe

#Columns required

organized_Usacovid_df = Usacovid_df[["date","state","positive","negative","death"]]

#Renaming columns

renamed_Usacovid_df = organized_Usacovid_df.rename(columns={"date":"start_date","state":"us_state","positive":"positive_cases","negative":"negative_cases","death":"number_of_death"})


#pd.to_datetime(renamed_Usacovid_df["start_date"])
# Keeping the dates greater and equal to 12feb2020

updated_dates_covid=renamed_Usacovid_df.loc[renamed_Usacovid_df["start_date"]>="2020/02/12",["start_date","us_state","positive_cases","negative_cases","number_of_death"]]

#Updated the date in ascending order
updated_df_covid = updated_dates_covid.sort_values(by = ["start_date"])

#Reseting the index
updated_df_covid = updated_df_covid.reset_index()

#Delete the index column
del updated_df_covid["index"]

#Modify NaN values to zero
updated_df_covid = updated_df_covid.fillna(0)

updated_df_covid.head()

    

,start_date,us_state,positive_cases,negative_cases,number_of_death
0,2020/02/28,WA,9.0,0.0,4.0
1,2020/02/29,WA,18.0,0.0,5.0
2,2020/03/01,RI,1.0,0.0,0.0
3,2020/03/01,MI,9.0,0.0,0.0
4,2020/03/01,WA,30.0,0.0,8.0


In [29]:
#Transforming the NYC STOCK exchange dataframe 

#Columns required

organized_stock_df = stock_df[["date","symbol","open","close","low","high"]]

#Renaming columns

rename_organized_stock_df = organized_stock_df.rename(columns={"date":"start_date","symbol":"symbol","open":"open_s","close":"close_s","low":"low","high":"high"})

# Keeping the dates greater and equal to 12feb2020

update_dates_stock = rename_organized_stock_df.loc[rename_organized_stock_df["start_date"]>="2020/02/12",["start_date","symbol","open_s","close_s","low","high"]]


#Drop all rows with missing information

updated_dates_stock = update_dates_stock.dropna(how="any")
updated_dates_stock.head()

#Reseting the index
updated_df_stock = updated_dates_stock.reset_index()

#Delete the index column
del updated_df_stock["index"]
updated_df_stock.head()

# Set new index to date

#df_stock=updated_dates_stock.set_index("Date")
#df_stock.head()

,start_date,symbol,open_s,close_s,low,high
0,2020/02/12,WLTW,108.559998,107.839996,107.070000,109.430000
1,2020/02/16,WLTW,109.110001,110.769997,107.010002,111.300003
2,2020/02/17,WLTW,110.830002,111.239998,107.970001,112.110001
3,2020/02/18,WLTW,111.120003,111.599998,108.930000,112.199997
4,2020/02/19,WLTW,111.370003,110.330002,107.279999,112.400002


In [30]:
merged_df = updated_df_covid.merge(updated_df_stock, how = 'inner')
merged_df.head()

,start_date,us_state,positive_cases,negative_cases,number_of_death,symbol,open_s,close_s,low,high
0,2020/02/29,WA,18.0,0.0,5.0,WLTW,113.040001,113.320000,111.650002,115.360001
1,2020/02/29,WA,18.0,0.0,5.0,A,37.590000,37.349998,37.330002,37.700001
2,2020/02/29,WA,18.0,0.0,5.0,AAL,40.770000,41.000000,40.500000,41.340000
3,2020/02/29,WA,18.0,0.0,5.0,AAP,150.000000,148.440002,148.380005,151.059998
4,2020/02/29,WA,18.0,0.0,5.0,AAPL,96.860001,96.690002,96.650002,98.230003


In [31]:
#Create data base

connection_string = "postgres:postgres@localhost:5432/ETL_db"  
engine = create_engine(f'postgresql://{connection_string}')

In [32]:
# Confirm tables
engine.table_names()

['stock', 'merged', 'covid']

In [33]:
updated_df_covid.to_sql(name='covid', con=engine, if_exists='append', index=False)

In [34]:
updated_df_stock.to_sql(name='stock', con=engine, if_exists='append', index=False)

In [35]:
merged_df.to_sql(name='merged', con=engine, if_exists='append', index=False)